# Check pnl across trades and positions

* Compare trade pnl between two subgraphs
* Compare position pnl between two subgraphs


In [14]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [15]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')

# mainnet
OLD_SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-perps'
SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/tburm/optimism-perps'
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# testnet
# SUBGRAPH_ENDPOINT = 'https://api.thegraph.com/subgraphs/name/kwenta/optimism-goerli-perps'
# RPC_ENDPOINT = f'https://optimism-kovan.infura.io/v3/{INFURA_KEY}'

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [16]:
# data contract
PerpsV2DataAbi = """
[
    {
      "inputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "_resolverProxy",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "nonpayable",
      "type": "constructor"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "allProxiedMarketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "globals",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "minInitialMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationFeeRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationBufferRatio",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minKeeperFee",
              "type": "uint256"
            }
          ],
          "internalType": "struct PerpsV2MarketData.FuturesGlobals",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        }
      ],
      "name": "marketDetails",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "marketDetailsForKey",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "baseAsset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "marketKey",
              "type": "bytes32"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxLeverage",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "maxMarketValue",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketLimits",
              "name": "limits",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "maxFundingVelocity",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "skewScale",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FundingParameters",
              "name": "fundingParameters",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "marketSize",
                  "type": "uint256"
                },
                {
                  "components": [
                    {
                      "internalType": "uint256",
                      "name": "long",
                      "type": "uint256"
                    },
                    {
                      "internalType": "uint256",
                      "name": "short",
                      "type": "uint256"
                    }
                  ],
                  "internalType": "struct PerpsV2MarketData.Sides",
                  "name": "sides",
                  "type": "tuple"
                },
                {
                  "internalType": "uint256",
                  "name": "marketDebt",
                  "type": "uint256"
                },
                {
                  "internalType": "int256",
                  "name": "marketSkew",
                  "type": "int256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.MarketSizeDetails",
              "name": "marketSizeDetails",
              "type": "tuple"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "price",
                  "type": "uint256"
                },
                {
                  "internalType": "bool",
                  "name": "invalid",
                  "type": "bool"
                }
              ],
              "internalType": "struct PerpsV2MarketData.PriceDetails",
              "name": "priceDetails",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "address[]",
          "name": "markets",
          "type": "address[]"
        }
      ],
      "name": "marketSummaries",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32[]",
          "name": "marketKeys",
          "type": "bytes32[]"
        }
      ],
      "name": "marketSummariesForKeys",
      "outputs": [
        {
          "components": [
            {
              "internalType": "address",
              "name": "market",
              "type": "address"
            },
            {
              "internalType": "bytes32",
              "name": "asset",
              "type": "bytes32"
            },
            {
              "internalType": "bytes32",
              "name": "key",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "price",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "marketSize",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "marketSkew",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "marketDebt",
              "type": "uint256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingRate",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "currentFundingVelocity",
              "type": "int256"
            },
            {
              "components": [
                {
                  "internalType": "uint256",
                  "name": "takerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFee",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "takerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "makerFeeOffchainDelayedOrder",
                  "type": "uint256"
                },
                {
                  "internalType": "uint256",
                  "name": "overrideCommitFee",
                  "type": "uint256"
                }
              ],
              "internalType": "struct PerpsV2MarketData.FeeRates",
              "name": "feeRates",
              "type": "tuple"
            }
          ],
          "internalType": "struct PerpsV2MarketData.MarketSummary[]",
          "name": "",
          "type": "tuple[]"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        }
      ],
      "name": "parameters",
      "outputs": [
        {
          "components": [
            {
              "internalType": "uint256",
              "name": "takerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "overrideCommitFee",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "takerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "makerFeeOffchainDelayedOrder",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxLeverage",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxMarketValue",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxFundingVelocity",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "skewScale",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "nextPriceConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "delayedOrderConfirmWindow",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "minDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "maxDelayTimeDelta",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMinAge",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "offchainDelayedOrderMaxAge",
              "type": "uint256"
            },
            {
              "internalType": "bytes32",
              "name": "offchainMarketKey",
              "type": "bytes32"
            },
            {
              "internalType": "uint256",
              "name": "offchainPriceDivergence",
              "type": "uint256"
            }
          ],
          "internalType": "struct IPerpsV2MarketSettings.Parameters",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "contract IPerpsV2MarketViews",
          "name": "market",
          "type": "address"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetails",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [
        {
          "internalType": "bytes32",
          "name": "marketKey",
          "type": "bytes32"
        },
        {
          "internalType": "address",
          "name": "account",
          "type": "address"
        }
      ],
      "name": "positionDetailsForMarketKey",
      "outputs": [
        {
          "components": [
            {
              "components": [
                {
                  "internalType": "uint64",
                  "name": "id",
                  "type": "uint64"
                },
                {
                  "internalType": "uint64",
                  "name": "lastFundingIndex",
                  "type": "uint64"
                },
                {
                  "internalType": "uint128",
                  "name": "margin",
                  "type": "uint128"
                },
                {
                  "internalType": "uint128",
                  "name": "lastPrice",
                  "type": "uint128"
                },
                {
                  "internalType": "int128",
                  "name": "size",
                  "type": "int128"
                }
              ],
              "internalType": "struct IPerpsV2MarketBaseTypes.Position",
              "name": "position",
              "type": "tuple"
            },
            {
              "internalType": "int256",
              "name": "notionalValue",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "profitLoss",
              "type": "int256"
            },
            {
              "internalType": "int256",
              "name": "accruedFunding",
              "type": "int256"
            },
            {
              "internalType": "uint256",
              "name": "remainingMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "accessibleMargin",
              "type": "uint256"
            },
            {
              "internalType": "uint256",
              "name": "liquidationPrice",
              "type": "uint256"
            },
            {
              "internalType": "bool",
              "name": "canLiquidatePosition",
              "type": "bool"
            }
          ],
          "internalType": "struct PerpsV2MarketData.PositionData",
          "name": "",
          "type": "tuple"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    },
    {
      "constant": true,
      "inputs": [],
      "name": "resolverProxy",
      "outputs": [
        {
          "internalType": "contract IAddressResolver",
          "name": "",
          "type": "address"
        }
      ],
      "payable": false,
      "stateMutability": "view",
      "type": "function"
    }
  ]
"""

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


In [17]:
# functions
def convertDecimals(x): return Decimal(x) / Decimal(10**18)


def convertBytes(x): return bytearray.fromhex(
    x[2:]).decode().replace('\x00', '')


def clean_df(df, decimal_cols=[], bytes_cols=[]):
    for col in decimal_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertDecimals)
        else:
            print(f"{col} not in DataFrame")
    for col in bytes_cols:
        if col in df.columns:
            df[col] = df[col].apply(convertBytes)
        else:
            print(f"{col} not in DataFrame")
    return df


async def run_query(query, params, endpoint=SUBGRAPH_ENDPOINT):
    transport = AIOHTTPTransport(url=endpoint)

    async with Client(
        transport=transport,
        fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        query = query

        result = await session.execute(query, variable_values=params)
        df = pd.DataFrame(result)
        return df


async def run_recursive_query(query, params, accessor, endpoint=SUBGRAPH_ENDPOINT):
  transport = AIOHTTPTransport(url=endpoint)

  async with Client(
      transport=transport,
      fetch_schema_from_transport=True,
  ) as session:
    done_fetching = False
    all_results = []
    while not done_fetching:
      result = await session.execute(query, variable_values=params)
      if len(result[accessor]) > 0:
        all_results.extend(result[accessor])
        params['last_id'] = all_results[-1]['id']
      else:
        done_fetching = True

    df = pd.DataFrame(all_results)
    return df


In [18]:
transfer_query = gql("""
query transfers(
    $last_id: ID!
  ) {
		futuresMarginTransfers(
      where: {
        id_gt: $last_id
      }
			first: 1000
		) {
			id
			timestamp
			account
			market
			size
			asset
			txHash
		}
}
""")
                     
trade_query = gql("""
query trades(
    $last_id: ID!
  ) {
    futuresTrades(
      where: {
        id_gt: $last_id
      }
      first: 1000
     ) {
      id
      account
      timestamp
      asset
      marketKey
      orderType
      price
      margin
      positionId
      feesPaid
      pnl
      size
      positionSize
      positionClosed
      trackingCode
    }  
}
""")


## Get transfers and trades

In [19]:
trade_params = {
    'last_id': ''
}

trade_decimal_cols = [
    'price',
    'margin',
    'feesPaid',
    'pnl',
    'size',
    'positionSize',
]

trade_bytes_cols = ['asset']

df_trade = await run_recursive_query(trade_query, trade_params, 'futuresTrades')
df_trade = df_trade.replace({None: np.nan})
df_trade = clean_df(df_trade, decimal_cols=trade_decimal_cols, bytes_cols=trade_bytes_cols)

In [20]:
transfer_params = {
    'last_id': ''
}

transfer_cols = [
    'size',
]

transfer_bytes_cols = [
    'asset'
]

df_transfer = await run_recursive_query(transfer_query, transfer_params, 'futuresMarginTransfers')
df_transfer = df_transfer.replace({None: np.nan})
df_transfer = clean_df(df_transfer, decimal_cols=transfer_cols, bytes_cols=transfer_bytes_cols)


In [21]:
## summarize transfers
df_transfer_summ = df_transfer.groupby('asset')['size'].sum().reset_index()
df_transfer_summ.columns = ['asset', 'netTransfers']

## summarize trades
df_trade_summ = df_trade.groupby('asset')['feesPaid'].sum().reset_index()
df_trade_summ.columns = ['asset', 'feesPaid']


In [22]:
df_trade_summ

,asset,feesPaid
0,AAVE,5094.591762314160759087
1,APE,30669.532999542737183659
2,ATOM,28130.091579015734492962
3,AUD,74.737962070163962639
4,AVAX,42849.898480165850638064
5,AXS,2649.541791782874909761
6,BNB,47217.162997526030553910
7,DOGE,10336.328727606199962510
8,DYDX,35583.552122562463601230
9,EUR,2041.927553550921278221


## Get the market data

In [24]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)
marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call()



In [25]:
markets = [{
    'asset': market[1].decode().replace('\x00', ''),
    'marketDebt': w3.fromWei(market[7], unit='ether')
} for market in marketSummaries]

df_markets = pd.DataFrame(markets)
df_markets

,asset,marketDebt
0,sETH,8287133.977944150633463063
1,sBTC,9229699.068605480573310876
2,LINK,179879.834105075521760298
3,SOL,185179.526789255126806185
4,AVAX,322992.648760464913796489
5,AAVE,2357.496780645747480827
6,UNI,79956.77687061548811295
7,MATIC,317082.168959722131612059
8,APE,56256.758894148802534243
9,DYDX,32725.580150335477993003


In [26]:
df_pnl = df_markets.merge(df_transfer_summ, on='asset').merge(df_trade_summ, on='asset')
df_pnl['netPnl'] = df_pnl['marketDebt'] - df_pnl['netTransfers'] - df_pnl['feesPaid']
df_pnl

,asset,marketDebt,netTransfers,feesPaid,netPnl
0,sETH,8287133.977944150633463063,9030884.506301814189766048,927714.756321998275062355,-1671465.284679661831365340
1,sBTC,9229699.068605480573310876,9590178.777201757778028972,784955.949143768318462261,-1145435.657740045523180357
2,LINK,179879.834105075521760298,273924.024376216487466098,91310.405407820622795708,-185354.595678961588501508
3,SOL,185179.526789255126806185,229580.430200758531091493,106168.080863121873284204,-150568.984274625277569512
4,AVAX,322992.648760464913796489,251832.826257212081153545,42849.898480165850638064,28309.924023086982004880
5,AAVE,2357.496780645747480827,372.107413050969437848,5094.591762314160759087,-3109.202394719382716108
6,UNI,79956.77687061548811295,93924.607307854175194074,10634.900755057813121603,-24602.731192296500202727
7,MATIC,317082.168959722131612059,404266.042059284001204993,84850.024781569465639001,-172033.897881131335231935
8,APE,56256.758894148802534243,30719.426208318101912868,30669.532999542737183659,-5132.200313712036562284
9,DYDX,32725.580150335477993003,52058.438858591547622279,35583.552122562463601230,-54916.410830818533230506


In [30]:
df_pnl['netPnl'].sum()

Decimal('-3925303.532448704920393727')

In [31]:
df_pnl['netPnlClean'] = df_pnl['netPnl'].apply(lambda x: round(x, 2))
df_pnl[['asset', 'netPnlClean']].sort_values('netPnlClean', ascending=True)

,asset,netPnlClean
0,sETH,-1671465.28
1,sBTC,-1145435.66
11,OP,-291173.94
2,LINK,-185354.60
7,MATIC,-172033.90
3,SOL,-150568.98
19,FTM,-121141.29
9,DYDX,-54916.41
16,ATOM,-45909.96
10,BNB,-45626.94
